In [1]:
# https://www.nltk.org/book/ch08.html
# Analyzing Sentence Structure

In [2]:
import nltk

In [3]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

In [4]:
sent = 'I shot an elephant in my pajamas'.split()

In [5]:
sent

['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

In [6]:
parser = nltk.ChartParser(groucho_grammar)

In [7]:
for tree in parser.parse(sent):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


In [8]:
grammar1 = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> "saw" | "ate" | "walked"
NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
Det -> "a" | "an" | "the" | "my"
N -> "man" | "dog" | "cat" | "telescope" | "park"
P -> "in" | "on" | "by" | "with"
""")

In [9]:
sent = "Mary saw Bob".split()

In [10]:
rd_parser = nltk.RecursiveDescentParser(grammar1)

In [11]:
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [12]:
nltk.app.srparser()

In [13]:
sr_parser = nltk.ShiftReduceParser(grammar1)

In [14]:
sent = 'Mary saw a dog'.split()

In [15]:
for tree in sr_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [1]:
text = 'I shot an elephant in my pajamas'.split()

In [5]:
groucho_grammar.productions(rhs=text[1])

[V -> 'shot']

In [6]:
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs()
    return wfst

In [16]:
def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1, nt2) in index:
                    wfst[start][end] = index[(nt1, nt2)]
                    if trace:
                        print('[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]' % \
                             (start, nt1, mid, nt2, end, start, index[(nt1, nt2)], end))
    return wfst

In [13]:
def display(wfst, tokens):
    print('\nWFST ' + ' '.join(('%-4d' % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print('%d  ' % i, end=' ')
        for j in range(1, len(wfst)):
            print('%-4s' % (wfst[i][j] or '.'), end=' ')
        print()

In [9]:
tokens = text

In [10]:
tokens

['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

In [11]:
wfst0 = init_wfst(tokens, groucho_grammar)

In [14]:
display(wfst0, tokens)


WFST 1    2    3    4    5    6    7   
0   NP   .    .    .    .    .    .    
1   .    V    .    .    .    .    .    
2   .    .    Det  .    .    .    .    
3   .    .    .    N    .    .    .    
4   .    .    .    .    P    .    .    
5   .    .    .    .    .    Det  .    
6   .    .    .    .    .    .    N    


In [17]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]


In [ ]:
nltk.app.chartparser()

/home/user/.virtualenvs/solenie/lib/python3.7/site-packages/nltk/app/__init__.py:46: UserWarning: nltk.app.wordfreq not loaded (requires the matplotlib library).
  "nltk.app.wordfreq not loaded " "(requires the matplotlib library)."


grammar= (
('    ', 'S -> NP VP,')
('    ', 'VP -> VP PP,')
('    ', 'VP -> V NP,')
('    ', 'VP -> V,')
('    ', 'NP -> Det N,')
('    ', 'NP -> NP PP,')
('    ', 'PP -> P NP,')
('    ', "NP -> 'John',")
('    ', "NP -> 'I',")
('    ', "Det -> 'the',")
('    ', "Det -> 'my',")
('    ', "Det -> 'a',")
('    ', "N -> 'dog',")
('    ', "N -> 'cookie',")
('    ', "N -> 'table',")
('    ', "N -> 'cake',")
('    ', "N -> 'fork',")
('    ', "V -> 'ate',")
('    ', "V -> 'saw',")
('    ', "P -> 'on',")
('    ', "P -> 'under',")
('    ', "P -> 'with',")
)
tokens = ['John', 'ate', 'the', 'cake', 'on', 'the', 'table']
Calling "ChartParserApp(grammar, tokens)"...
